<a href="https://colab.research.google.com/github/Ansuman09/cv/blob/main/REMY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json


## REMY

Creating the json file

In [ ]:
json_dict={'intents':[{
    'tag': 'hello',
    'patterns':['you up?','Hello','Hi','Heya','Hola','Hey there','Is any one there?'],
    'responses':['Pleased to hear from you, Sir','I am here how can i assist you sir']
},
  {
    'tag': 'goodbye',
    'patterns': ['lets talk about it later','see you later','Goodbye','Bye'],
    'responses':['Okay','I will be here if you need me','Have a great day']
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me",'Nice talking to you'],
     "responses": ["Happy to help!", "Any time!", "My pleasure",'You are welcome'],
     },
    {'tag':'jokes',
     'patterns':['tell me a joke','tell me something i dont know','How smart are you'],
     'responses':['Why did the programmers act so nice to the AI? Because it was really self conscious.',
                  'If an AI simulation of a pop singer performs all over the world... Does that mean she passes the touring test?',
                  'Don’t you hate it when someone answers their own questions? I do.',
                  'I used to think I was indecisive. But now I’m not so sure.',
                  'The future, the present, and the past walk into a bar. Things got a little tense.']},
    {"tag": "options",
    "patterns": ["How you could help me?", "How you can be helpful?", "What support is offered",'What can you do'],
    "responses": ["Inform you about the items at store such as availability, price and take orders... to place orders, visit this page"],
    },
    {'tag':'make_orders',
     'patterns':['I want to make an order','order this item'],
     'responses':['Very well, click on this link to place your order']
    },
    {'tag':'delivery',
     'patterns':['Do you make home delivery?','Deliver goods at home','Deliver copies at home'],
     'responses':['I am afraid we dont provide such services as of now']
    },
    {'tag':'occassion',
     'patterns':['What gift items do you have?','Suggest a gift item','occassion','what birthday gifts do you have'],
     'responses':['We have gift envelopes, pencilboxes, Crayons, partypoppers, choclates and many more']  
    },
    {'tag':'available',
     'patterns':['What kind of items do you have','What services do you provide','Do you have school utensils','What is available'],
     'responses':['We have plain copies, ruled copies, paper, graphs, maps, charts and other project and study utensils in the shop']  
    },
    {'tag':'prices',
     'patterns':['What does it cost?','How much is it','price?','Is it expensive'],
     'responses':['The prices are reasonable','We offer the best price in the market']}
    ]}
with open('intents.json','w') as file:
  json.dump(json_dict,file)

Let's Import the Dependencies,

we use nltk to tokenize the text and use the torch framework for building the NeuralNet

In [ ]:
from nltk.stem.porter import PorterStemmer
import numpy as np
import torch 
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import urllib
import nltk
nltk.download('punkt')
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## The Neural Network

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    """
    this function will take the input size i.e. the size of the arry, 
    the hidden layer size and output size that is the size of the labels array
    and output a neural net
    """
    super(NeuralNet,self).__init__()
    self.l1=nn.Linear(input_size,hidden_size)
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,output_size)
    self.relu=nn.ReLU()

  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    return out

use porter stemmmer to stem words for example words like ..
['go','gone','going'] could be considered as a single word 'go',

In [ ]:
stemmer=PorterStemmer()

def tokenizer(sentence):
  return nltk.word_tokenize(sentence)

def stem_w(word):
  return stemmer.stem(word.lower())


for n in tokenizer('i like to eat liking ice-cream'):
  print(stem_w(n))

def bag_of_words(tokenized_sentence,all_words):
  tokenized_sentence=[stem_w(word) for word in tokenized_sentence]

  bag=np.zeros(len(all_words),dtype=np.float32)
  for idx,word in enumerate(all_words):
    if word in tokenized_sentence:
      bag[idx]=1
  return bag

#the words like and liking are treated as the same i.e. like

i
like
to
eat
like
ice-cream


The following code creates a bag of words that would be later used to mask each of the sentence. If our bag of words contain... ['wake', 'coffee'] then the mask we have implemented later would create an array for a givn line: 'I will wake up and make some cofffee' ..as [0 0 1 0 0 0 0 1] the values one means that the word used in the following sentence was in the bag of words. 

In [ ]:


url = 'intents.json'

with open(url, 'r') as f:
    intents = json.load(f)

all_words=[]
tags=[]
xy=[]

for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenizer(pattern)
    all_words.extend(w)
    xy.append((w,tag))

In [ ]:
ignore_words=['?','!','.',',']
all_words=[stem_w(word) for word in all_words if word not in ignore_words]

all_words=sorted(set(all_words))
tags=sorted(set(tags))

In [ ]:
#creating the train set and labels the labels are the tags in intents.json created earlier
X_train=[]
y_train=[]
for tokenized_text,tag in xy:
  bag=bag_of_words(tokenized_text,all_words)
  X_train.append(bag)
  
  label=tags.index(tag)
  y_train.append(label)

X_train=np.array(X_train)
y_train=np.array(y_train)

In [ ]:
X_train[0].shape

(74,)

This is how the bag of words works

In [ ]:

print(f'Words dict       : {all_words}')
print(f'tokenized_words  : {xy[0][0]},{xy[1][0]},{xy[2][0]}')
print(f'train_set        : {X_train[:5]}')
print(f'labels           : {y_train[:3]}')


Words dict       : ["'s", 'a', 'about', 'an', 'ani', 'are', 'at', 'avail', 'awesom', 'be', 'birthday', 'bye', 'can', 'copi', 'cost', 'could', 'deliv', 'deliveri', 'do', 'doe', 'dont', 'expens', 'for', 'gift', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'heya', 'hi', 'hola', 'home', 'how', 'i', 'is', 'it', 'item', 'joke', 'kind', 'know', 'later', 'let', 'make', 'me', 'much', 'nice', 'occass', 'of', 'offer', 'one', 'order', 'price', 'provid', 'school', 'see', 'servic', 'smart', 'someth', 'suggest', 'support', 'talk', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'up', 'utensil', 'want', 'what', 'you']
tokenized_words  : ['you', 'up', '?'],['Hello'],['Hi']
train_set        : [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.

creating a torch dataset to use our train set

In [ ]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=y_train 
  
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  
  def __len__(self):
    return self.n_samples

## Training the model

In [ ]:
batch_size=8
hidden_dim=8
output_dim=len(tags)
input_dim=len(X_train[0])
learning_rate=0.01
num_epochs=1000

dataset=ChatDataset()
batched_data=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=2)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NeuralNet(input_dim,hidden_dim,output_dim)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

for epoch in range(num_epochs):
  for (words,labels) in batched_data:
    words=words.to(device)
    labels=labels.to(device)

    output=model(words)
    loss=criterion(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1)%100 == 0:
    print(f'epoch {epoch+1}/{num_epochs}, Loss is={loss}')

print("Final Loss, loss{:.4f}".format(loss.item()))

epoch 100/1000, Loss is=0.000350175570929423
epoch 200/1000, Loss is=9.42901024245657e-05
epoch 300/1000, Loss is=4.434487345861271e-05
epoch 400/1000, Loss is=5.722029527532868e-06
epoch 500/1000, Loss is=9.65590606938349e-06
epoch 600/1000, Loss is=3.790783375734463e-05
epoch 700/1000, Loss is=1.5497195136049413e-06
epoch 800/1000, Loss is=1.3112935448589269e-05
epoch 900/1000, Loss is=1.0967194612021558e-05
epoch 1000/1000, Loss is=1.1920922133867862e-06
Final Loss, loss0.0000


## Saving the model as data.pth to use in the tk inter app

In [ ]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_dim,
    "output_size":output_dim,
    "hiddent_size": hidden_dim,
    "all_words":all_words,
    "tags": tags
}
FILE = "data.pth"
torch.save(data, FILE)

print("Traning complete. file saved to",FILE)

Traning complete. file saved to data.pth
